In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import spacy

**Dataset**

In [2]:
data = pd.read_excel('../data/news_dataset.xlsx')

In [3]:
data.head()

,text,label
0,No comment is expected from Barack Obama Membe...,fake
1,Did they post their votes for Hillary already?,fake
2,"Now, most of the demonstrators gathered last ...",fake
3,A dozen politically active pastors came here f...,real
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",fake


## Data Preprocessing

In [6]:
nlp = spacy.load("en_core_web_sm")
text = "Tokenize this sentence using Spacy."
doc = nlp(text)
[token.text for token in doc]

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.